In [ ]:
!pip install yellowbrick
import yellowbrick
from pathlib import Path
import pendulum
import dill
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import clone 
from sklearn.feature_selection import RFECV
import seaborn as sns
import xgboost as xgb
import missingno as msno
import pandas as pd
from xgboost import XGBRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor  
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder, OneHotEncoder
import shap
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from scipy.stats import loguniform, uniform
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from loguru import logger
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, GradientBoostingRegressor
pd.set_option("display.max_columns", None)


warnings.filterwarnings('ignore')


In [ ]:
# 1. Chargement et analyse initiale des données
def load_and_analyze_data(file_path):
    """
    Charge les données et effectue une analyse initiale
    """
   
    data = pd.read_csv(file_path, na_values=['?'])
    data.columns = data.columns.str.replace("-", "_")

    print("Informations sur le dataset:")
    print(data.info())
    print("\nAperçu des 5 premières lignes:")
    print(data.head())
    print("\nStatistiques descriptives:")
    print(data.describe(include='all'))
    
    missing_values = data.isnull().sum()
    print("\nValeurs manquantes par colonne:")
    print(missing_values[missing_values > 0])
    
    return data

In [ ]:
# 3. Analyse exploratoire des données (EDA)
def perform_eda(data):
    """
    Réalise une analyse exploratoire des données
    """
    # Distribution du prix
    plt.figure(figsize=(10, 6))
    sns.histplot(data['price'].dropna(), kde=True)
    plt.title('Distribution du prix des voitures')
    plt.xlabel('Prix')
    plt.ylabel('Fréquence')
    plt.savefig('price_distribution.png')
    plt.close()
    
    # Relation entre variables numériques et prix
    numerical_features = ['wheel_base', 'length', 'width', 'height', 'curb_weight', 
                         'engine_size', 'bore', 'stroke', 'compression_ratio', 
                         'horsepower', 'peak_rpm','symboling', 'city_mpg', 'highway_mpg']
    
    plt.figure(figsize=(15, 10))
    for i, feature in enumerate(numerical_features[:9]):
        plt.subplot(3, 3, i+1)
        plt.scatter(data[feature], data['price'], alpha=0.5)
        plt.title(f'Prix vs {feature}')
        plt.xlabel(feature)
        plt.ylabel('Prix')
    plt.tight_layout()
    plt.savefig('price_vs_features.png')
    plt.close()
    
    # Matrice de corrélation
    numeric_df = data.select_dtypes(include=['int64', 'float64'])
    plt.figure(figsize=(12, 10))
    correlation_matrix = numeric_df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Matrice de corrélation des variables numériques')
    plt.savefig('correlation_matrix.png')
    plt.close()
    
    # Distribution des prix par marque
    plt.figure(figsize=(14, 8))
    brand_avg_price = data.groupby('make')['price'].mean().sort_values(ascending=False)
    sns.barplot(x=brand_avg_price.index, y=brand_avg_price.values)
    plt.title('Prix moyen par marque')
    plt.xticks(rotation=90)
    plt.xlabel('Marque')
    plt.ylabel('Prix moyen')
    plt.savefig('avg_price_by_brand.png')
    plt.close()
    
    return data, correlation_matrix

In [ ]:
# 4. Traitement des valeurs manquantes et aberrantes
def handle_missing_and_outliers(data, price_col='price', threshold=0.7, show_plots=True):
    """
    Traite les valeurs manquantes et aberrantes de manière complète
    
    Args:
        data (pd.DataFrame): DataFrame contenant les données
        price_col (str): Nom de la colonne cible (prix)
        threshold (float): Seuil pour supprimer les colonnes trop manquantes (0-1)
        show_plots (bool): Afficher les visualisations si True
    
    Returns:
        pd.DataFrame: DataFrame nettoyé
        dict: Statistiques de nettoyage
    """
    # Initialisation des statistiques
    stats = {
        'missing_before': data.isnull().sum().sum(),
        'outliers_detected': {},
        'columns_dropped': []
    }
    
    # 1. Visualisation des valeurs manquantes
    if show_plots:
        plt.figure(figsize=(15, 5))
        msno.matrix(data)
        plt.title('Distribution des Valeurs Manquantes', pad=20)
        plt.show()
    
    # 2. Suppression des colonnes avec trop de valeurs manquantes
    cols_to_drop = data.columns[data.isnull().mean() > threshold]
    if not cols_to_drop.empty:
        stats['columns_dropped'] = cols_to_drop.tolist()
        data = data.drop(columns=cols_to_drop)
        print(f"Colonnes supprimées (>{threshold*100}% manquants): {list(cols_to_drop)}")

    
    # 3. Détection et traitement des outliers pour toutes les variables numériques
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    
    for col in numeric_cols:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        
        # Détection des outliers
        outliers_mask = (data[col] < lower) | (data[col] > upper)
        n_outliers = outliers_mask.sum()
        
        if n_outliers > 0:
            stats['outliers_detected'][col] = {
                'count': n_outliers,
                'percentage': n_outliers / len(data) * 100,
                'method': 'IQR'
            }
            
            # Remplacement des outliers par NaN
            data[col] = np.where(outliers_mask, np.nan, data[col])
    
    # 4. Traitement spécial pour la variable cible (price)
    if price_col in data.columns:
        q1 = data[price_col].quantile(0.25)
        q3 = data[price_col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Visualisation des outliers pour price
        if show_plots:
            plt.figure(figsize=(10, 6))
            sns.boxplot(x=data[price_col].dropna())
            plt.title('Distribution du prix avec limites des outliers', pad=15)
            plt.axvline(lower_bound, color='r', linestyle='--', label='Limite inférieure')
            plt.axvline(upper_bound, color='r', linestyle='--', label='Limite supérieure')
            plt.legend()
            plt.show()
            
            # Boxplot pour toutes les variables numériques
            plt.figure(figsize=(15, 8))
            sns.boxplot(data=data[numeric_cols])
            plt.xticks(rotation=45)
            plt.title('Distribution des variables numériques après traitement', pad=15)
            plt.tight_layout()
            plt.show()
    
    # 5. Statistiques finales
    stats['missing_after'] = data.isnull().sum().sum()
    stats['remaining_columns'] = list(data.columns)
    
    print(f"\nRésumé du nettoyage:")
    print(f"- Valeurs manquantes initiales: {stats['missing_before']}")
    print(f"- Valeurs manquantes après traitement: {stats['missing_after']}")
    print(f"- Colonnes supprimées: {stats['columns_dropped'] or 'Aucune'}")
    print("\nDétection des outliers:")
    for col, info in stats['outliers_detected'].items():
        print(f"- {col}: {info['count']} outliers ({info['percentage']:.2f}%)")
    
    # Sauvegarde des données nettoyées
    data['price_original'] = data.get(price_col, np.nan)
    return data, stats